### Imports

In [1]:
from __future__ import division, print_function
from collections import OrderedDict

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt
import time
import os
import cv2
from PIL import Image, ImageChops

plt.ion()   # interactive mode
%matplotlib inline
%env TMP="/tmp"
%env JOBLIB_TEMP_FOLDER="/tmp"

env: TMP="/tmp"
env: JOBLIB_TEMP_FOLDER="/tmp"


In [2]:
def summary(input_size, model):
    def register_hook(module):
        def hook(module, input, output):
            class_name = str(module.__class__).split('.')[-1].split("'")[0]
            module_idx = len(summary)

            m_key = '%s-%i' % (class_name, module_idx+1)
            summary[m_key] = OrderedDict()
            summary[m_key]['input_shape'] = list(input[0].size())
            summary[m_key]['input_shape'][0] = -1
            summary[m_key]['output_shape'] = list(output.size())
            summary[m_key]['output_shape'][0] = -1

            params = 0
            if hasattr(module, 'weight'):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                if module.weight.requires_grad:
                    summary[m_key]['trainable'] = True
                else:
                    summary[m_key]['trainable'] = False
            if hasattr(module, 'bias'):
                params +=  torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]['nb_params'] = params

        if not isinstance(module, nn.Sequential) and \
           not isinstance(module, nn.ModuleList) and \
           not (module == model):
            hooks.append(module.register_forward_hook(hook))

    # check if there are multiple inputs to the network
    if isinstance(input_size[0], (list, tuple)):
        x = [Variable(torch.rand(1,*in_size)) for in_size in input_size]
    else:
        x = Variable(torch.rand(1,*input_size))

    # create properties
    summary = OrderedDict()
    hooks = []
    # register hook
    model.apply(register_hook)
    # make a forward pass
    model(x)
    # remove these hooks
    for h in hooks:
        h.remove()

    return summary

### Data

In [3]:
TRAIN_DIR = '../data/boneage-training-dataset/'
TRAIN_LABEL = '../data/train.csv'
FILETYPE = '.png'
use_gpu = torch.cuda.is_available()

### Dataset class

In [4]:
class BoneAgeDataset(Dataset):
    """Bone Age dataset."""

    def __init__(self, root_dir, labels, filetype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.filetype = filetype

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.labels.iloc[idx]['id']) + self.filetype)
        img = Image.open(img_name).convert('RGB')
#         image = cv2.imread(img_name)[:,:,::-1]
        boneage = self.labels.iloc[idx]['boneage'].astype(np.float32)
        gender = self.labels.iloc[idx]['male'].astype(np.uint8)
        
        label = np.hstack((boneage, gender))

        if self.transform:
            img = self.transform(img)
            
        sample = {'image': img, 'label': label}

        return sample

### Transforms

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(1024),
        transforms.CenterCrop(512),
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Scale(1024),
#         transforms.CenterCrop(512),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

### Data loaders

In [6]:
labels = pd.read_csv(TRAIN_LABEL)
bins = xrange(20)
labels['bins'] = np.digitize(labels['boneage'], bins=bins)

In [7]:
skf = StratifiedKFold(n_splits=4)
splits = []
for train, test in skf.split(np.zeros(len(labels)), labels['bins']):
    fold_label = {
        'train': labels.loc[train, ('id', 'boneage', 'male')],
        'val': labels.loc[test, ('id', 'boneage', 'male')]
    }
    splits.append(fold_label)

/home/user/software/miniconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


In [8]:
fold_label = splits[0]

In [9]:
image_datasets = {x: BoneAgeDataset(TRAIN_DIR, fold_label[x], FILETYPE, data_transforms[x])
                  for x in ['train', 'val']}
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8, shuffle=True, num_workers=6, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [10]:
for i_batch, sample in enumerate(dataloders['val']):
    print(i_batch, sample['image'].size())
    print(i_batch, sample['image'].mean())
    if i_batch == 2:
        break

0 torch.Size([8, 3, 224, 224])
0 -0.824975125545
1 torch.Size([8, 3, 224, 224])
1 -0.714101304884
2 torch.Size([8, 3, 224, 224])
2 -0.959426448647


### Training

In [11]:
class Net(nn.Module):
    def __init__(self, input_shape=(3, 224, 224)):
        super(Net, self).__init__()
        
        self.bn = nn.BatchNorm2d(input_shape[0])
        resnet = models.resnet18(pretrained=True)
        num_ftrs = resnet.fc.in_features
        resnet.fc = nn.Linear(num_ftrs, 2)

        self.resnet = resnet

    def forward(self, x):
        x = self.resnet(self.bn(x))
        return x
    
model = Net([3, 224, 224])
    
# print(summary([3, 224, 224], model))

if use_gpu:
    model.cuda()

In [12]:
lr = 0.001
momentum = 0.9
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), lr=lr)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [13]:
def ccc(x, y):
#     print('X:', x)
    mean_x = torch.mean(x)
#     print('Mean x:', mean_x)
    mean_y = torch.mean(y)
#     print('Mean y:', mean_y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    xn = torch.norm(xm, 2)
#     print('Xn:', xn)
    yn = torch.norm(ym, 2)
#     print('Yn:', yn)
    xydot = ym.dot(xm.view(-1))
#     print('XYdot:', xydot)

    ccc_num = 2 * xydot / xm.size()[0]
#     print('CCC num:', ccc_num)
    ccc_den = xn + yn + torch.pow(mean_x.sub(mean_y), 2)
#     print('CCC den:', ccc_den)

    ccc = ccc_num / ccc_den
    return ccc
    

def train(epoch, train_loader):
    exp_lr_scheduler.step()
    model.train()
    train_boneage_loss = 0
    train_gender_loss = 0
    train_loss = 0
    train_ccc = 0
    for batch_idx, sample in enumerate(train_loader):
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        boneage_loss = F.l1_loss(output[:,0], target[:,0])
        train_boneage_loss += boneage_loss.data[0]
        gender_loss = F.binary_cross_entropy(F.sigmoid(output[:,1]), target[:,1])
        train_gender_loss += gender_loss.data[0]
        loss = 2 * boneage_loss + gender_loss
        train_loss += loss.data[0]
        c = ccc(output[:,0], target[:,0])
        train_ccc += c.data.cpu().numpy()[0]
        loss.backward()
        optimizer.step()
#         if batch_idx % (1000 // len(target)) == 0:
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBoneAge Loss: {:.4f}\tGender Loss: {:.4f}\t Loss: {:.4f}\tCCC: {:.4f}'.format(
        epoch, batch_idx * len(data), len(train_loader),
        100. * batch_idx / len(train_loader), train_boneage_loss / len(train_loader),
        train_gender_loss / len(train_loader), train_loss / len(train_loader),
        train_ccc / len(train_loader)))

def test(test_loader):
    model.eval()
    test_boneage_loss = 0
    test_gender_loss = 0
    test_loss = 0
    test_ccc = 0
    for sample in test_loader:
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        boneage_loss = F.l1_loss(output[:,0], target[:,0])
        test_boneage_loss += boneage_loss.data[0]
        gender_loss = F.binary_cross_entropy(F.sigmoid(output[:,1]), target[:,1])
        test_gender_loss += gender_loss.data[0]
        loss = 2 * boneage_loss + gender_loss
        test_loss += loss.data[0]
        c = ccc(output[:,0], target[:,0])
        test_ccc += c.data.cpu().numpy()[0]

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Avg BoneAge loss: {:.4f}, Avg Gender loss: {:.4f}, Avg loss: {:.4f} CCC: {:.4f}\n'.format(
        test_boneage_loss / len(test_loader), test_gender_loss / len(test_loader),
        test_loss / len(test_loader), test_ccc / len(test_loader)))

In [14]:
# torch.save(model.state_dict(), '../models/9-27-1540-resnet50-1fold-5.7889-2.56.pth')

In [ ]:
for epoch in range(1, 40 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

Train Epoch: 14 [5905/1182 (100%)]	BoneAge Loss: 15.8622	Gender Loss: 0.7003	 Loss: 32.4247	CCC: 8.1534

Test set: Avg BoneAge loss: 36.4150, Avg Gender loss: 0.7005, Avg loss: 0.0233 CCC: 2.6063

Train Epoch: 15 [5905/1182 (100%)]	BoneAge Loss: 15.5458	Gender Loss: 0.6979	 Loss: 31.7895	CCC: 8.3220

Test set: Avg BoneAge loss: 26.9706, Avg Gender loss: 0.7039, Avg loss: 0.0173 CCC: 4.8107

Train Epoch: 16 [5905/1182 (100%)]	BoneAge Loss: 15.4276	Gender Loss: 0.6988	 Loss: 31.5541	CCC: 8.3059

Test set: Avg BoneAge loss: 45.4811, Avg Gender loss: 0.7010, Avg loss: 0.0290 CCC: 1.4971

Train Epoch: 17 [5905/1182 (100%)]	BoneAge Loss: 15.4484	Gender Loss: 0.6944	 Loss: 31.5911	CCC: 8.3387

Test set: Avg BoneAge loss: 39.7256, Avg Gender loss: 0.7599, Avg loss: 0.0254 CCC: 2.4498

Train Epoch: 18 [5905/1182 (100%)]	BoneAge Loss: 15.1520	Gender Loss: 0.6984	 Loss: 31.0025	CCC: 8.5418

Test set: Avg BoneAge loss: 30.2005, Avg Gender loss: 0.7014, Avg loss: 0.0193 CCC: 4.1793

Train Epoch: 19